In [1]:
!pip install transformers[torch] -U

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

In [10]:
def fine_tune_model(dataset_path, model_name="gpt2-medium", output_dir="./gpt2_finetuned"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Set the tokenizer's padding token if necessary
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Load and preprocess the dataset
    dataset = load_dataset('text', data_files={'train': dataset_path})

    def tokenize_function(examples):
    # Tokenize the text and ensure that the attention_mask is included
      tokenized_inputs = tokenizer(
        examples['text'],
        padding=True,  # Activates padding
        truncation=True,
        max_length=512,
        return_attention_mask=True  # Ensure attention_mask is returned
      )
      tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
      return tokenized_inputs

    tokenized_dataset = dataset.map(tokenize_function, batched=True)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=1,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset['train'],
    )

    # Fine-tune the model
    trainer.train()

    # Save the fine-tuned model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

if __name__ == "__main__":
    fine_tune_model('dataset.txt')


Map:   0%|          | 0/87 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model(model_dir="gpt2_finetuned"):
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForCausalLM.from_pretrained(model_dir)
    return model, tokenizer

def generate_text(model, tokenizer, prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def interactive_storytelling(model, tokenizer):
    story = input("Welcome: ")
    print("\nStory so far: " + story)

    while True:
        user_input = input("\nYour input to shape the story: ")
        if user_input.lower() == 'exit':
            break

        prompt = story + " " + user_input
        story_continuation = generate_text(model, tokenizer, prompt)
        story += story_continuation
        print("\nStory so far: " + story)

if __name__ == "__main__":
    model, tokenizer = load_model()
    interactive_storytelling(model, tokenizer)
